# IMPORTAÇÃO DAS BIBLIOTECAS

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window as W

# CRIAÇÃO DA SPARK SESSION

In [ ]:
spark = (SparkSession.builder
        .master("local")
        .appName("Pyspark_SP")
        .config("spark.ui.port", "4050")
        .getOrCreate()
        )

In [3]:
spark

# EXTRAÇÃO DE DADOS DE SP

In [24]:
df_SP = spark.read.parquet("gs://projeto_final_2021/pandas_to_parquet/SP_parquet/df_SP.snappy")

In [25]:
df_SP.show()

+--------+---+----+--------------------+--------------------+--------------+-------------------+----------+-----------+--------+----------+-------------------+--------------------+--------------------+-------+
| EMPRESA|MÊS| ANO|                NOME|               CARGO|       LOTAÇÃO|           ADMISSÃO|NASCIMENTO|VENCIMENTOS|ENCARGOS|BENEFÍCIOS|OUTRAS_REMUNERAÇÕES|             VÍNCULO|     DETALHE_VÍNCULO|LIMINAR|
+--------+---+----+--------------------+--------------------+--------------+-------------------+----------+-----------+--------+----------+-------------------+--------------------+--------------------+-------+
|ADESAMPA|  1|2020|ALESSANDRA SERAPO...| Sup. Adm Financeiro|          SEDE|2019-02-06 00:00:00|      1974|    5964.83| 2656.28|    625.01|                0.0|CLT: contrato por...|Não concursado oc...|   null|
|ADESAMPA|  1|2020|  ALINE DIAS MARTINS|          Analista I|          SEDE|2018-04-02 00:00:00|      1987|    4906.81|  2152.8|    777.26|                0.0|C

In [26]:
df_SP.printSchema()

root
 |-- EMPRESA: string (nullable = true)
 |-- MÊS: byte (nullable = true)
 |-- ANO: short (nullable = true)
 |-- NOME: string (nullable = true)
 |-- CARGO: string (nullable = true)
 |-- LOTAÇÃO: string (nullable = true)
 |-- ADMISSÃO: timestamp (nullable = true)
 |-- NASCIMENTO: short (nullable = true)
 |-- VENCIMENTOS: float (nullable = true)
 |-- ENCARGOS: float (nullable = true)
 |-- BENEFÍCIOS: float (nullable = true)
 |-- OUTRAS_REMUNERAÇÕES: float (nullable = true)
 |-- VÍNCULO: string (nullable = true)
 |-- DETALHE_VÍNCULO: string (nullable = true)
 |-- LIMINAR: string (nullable = true)



# ORDENANDO, CRIANDO E ALTERANDO COLUNAS

Etapa importante para a futura concatenação de ambos os estados.

In [27]:
df_SP.schema.names

['EMPRESA',
 'MÊS',
 'ANO',
 'NOME',
 'CARGO',
 'LOTAÇÃO',
 'ADMISSÃO',
 'NASCIMENTO',
 'VENCIMENTOS',
 'ENCARGOS',
 'BENEFÍCIOS',
 'OUTRAS_REMUNERAÇÕES',
 'VÍNCULO',
 'DETALHE_VÍNCULO',
 'LIMINAR']

## CRIAÇÃO DA COLUNA SALARIOLIQUIDO

In [28]:
df_SP = df_SP.fillna(0, subset=['BENEFÍCIOS', 'ENCARGOS'])
df_SP = df_SP.withColumn("salarioLiquido",col("VENCIMENTOS") + col("BENEFÍCIOS") - col("ENCARGOS"))

## FILTRANDO COLUNAS NECESSÁRIAS PARA ANÁLISE

Nesta etapa foi necessário criar a coluna de estado para futuras análises

In [29]:
coletar_colunas = {'MÊS','ANO','NOME','CARGO','VÍNCULO','salarioLiquido'}

remover_colunas = list(set(df_SP.schema.names).difference(coletar_colunas))

valor = 'SP'

df_novo = df_SP.drop(*remover_colunas).withColumn('Estado', lit(valor).cast('string'))

## PADRONIZANDO NOME DAS COLUNAS

In [30]:
#Alterar nome da coluna
df_novo = (df_novo.withColumnRenamed('MÊS', 'mes')
        .withColumnRenamed('ANO', 'ano')
        .withColumnRenamed('NOME', 'nome')
        .withColumnRenamed('CARGO', 'cargo')
        .withColumnRenamed('VÍNCULO', 'situacao')
        .withColumnRenamed('Estado', 'estado')
)

## PADRONIZANDO DADOS DA COLUNA SITUACAO

In [31]:
df_novo.show()

+---+----+--------------------+--------------------+--------------------+--------------+------+
|mes| ano|                nome|               cargo|            situacao|salarioLiquido|estado|
+---+----+--------------------+--------------------+--------------------+--------------+------+
|  1|2020|ALESSANDRA SERAPO...| Sup. Adm Financeiro|CLT: contrato por...|     3933.5598|    SP|
|  1|2020|  ALINE DIAS MARTINS|          Analista I|CLT: contrato por...|     3531.2703|    SP|
|  1|2020| ANA MARIA M. CAIRES|Gestor(a) de Recu...|CLT: contrato por...|       2947.29|    SP|
|  1|2020|ANA PAULA DE BARR...|Sup. Relações Ins...|CLT: contrato por...|     6716.3906|    SP|
|  1|2020|ARMANDO DE ALMEID...|        Assessor III|CLT: contrato por...|     5211.8105|    SP|
|  1|2020|   ARY SCAPIN JUNIOR| Gerente de Projetos|CLT: contrato por...|      5654.839|    SP|
|  1|2020|BIANCA M. M. N. S...|         Analista II|CLT: contrato por...|       3971.21|    SP|
|  1|2020|  CAROLINA S. ATTILI|Agente de

In [38]:
df_novo.filter(col("situacao").isNull()).show()

+---+---+----+-----+--------+--------------+------+
|mes|ano|nome|cargo|situacao|salarioLiquido|estado|
+---+---+----+-----+--------+--------------+------+
+---+---+----+-----+--------+--------------+------+



In [35]:
df_novo = (df_novo.withColumn("situacao",
                             when(col("situacao") == "Desligado", "INATIVO")
                             .when(col("situacao") == "Aposentado", "INATIVO")
                             .when(col("situacao").isNull(), "OUTRO")
                             .otherwise("ATIVO")
                             )
          )

## ORDENANDO COLUNAS

In [39]:
df_novo.schema.names

['mes', 'ano', 'nome', 'cargo', 'situacao', 'salarioLiquido', 'estado']

In [40]:
df_novo = df_novo.select('mes','ano','nome','cargo','situacao','salarioLiquido','estado')

# VISUALIZAÇÃO TABELA FINAL DE SP

In [41]:
df_novo.show()

+---+----+--------------------+--------------------+--------+--------------+------+
|mes| ano|                nome|               cargo|situacao|salarioLiquido|estado|
+---+----+--------------------+--------------------+--------+--------------+------+
|  1|2020|ALESSANDRA SERAPO...| Sup. Adm Financeiro|   ATIVO|     3933.5598|    SP|
|  1|2020|  ALINE DIAS MARTINS|          Analista I|   ATIVO|     3531.2703|    SP|
|  1|2020| ANA MARIA M. CAIRES|Gestor(a) de Recu...|   ATIVO|       2947.29|    SP|
|  1|2020|ANA PAULA DE BARR...|Sup. Relações Ins...|   ATIVO|     6716.3906|    SP|
|  1|2020|ARMANDO DE ALMEID...|        Assessor III|   ATIVO|     5211.8105|    SP|
|  1|2020|   ARY SCAPIN JUNIOR| Gerente de Projetos|   ATIVO|      5654.839|    SP|
|  1|2020|BIANCA M. M. N. S...|         Analista II|   ATIVO|       3971.21|    SP|
|  1|2020|  CAROLINA S. ATTILI|Agente de Desenvo...|   ATIVO|     2404.8398|    SP|
|  1|2020|CINTIA G. SIQUEIR...|       Assistente II|   ATIVO|     2781.5103|

# EXPORTAÇÃO DO ARQUIVO

In [42]:
(df_novo.write.format("parquet")
.option("header", "true")
.option("inferschema", "true")
.save("gs://projeto_final_2021/parquet_to_bq/SP")
)